# Tutorial: Pseudobulk eQTL Analysis with `cellink`

This tutorial demonstrates how to perform pseudobulk eQTL analysis using the `cellink` package, which integrates single-cell transcriptomics with genotype data. We focus on one cell type and test for cis-eQTLs across the genome, illustrating key steps such as pseudobulk aggregation, dimensionality reduction, filtering, and linear modeling.

This notebook assumes familiarity with single-cell data processing and basic statistical genetics concepts (e.g., eQTLs, PCA). If you're new to `cellink`, we recommend reviewing the [documentation page](#) for a conceptual overview of the package.

## Environment Setup

We begin by importing necessary libraries and defining key parameters for our analysis. `cellink` provides utilities that extend `AnnData` to handle both donor-level genotype data and single-cell RNA-seq data efficiently.

In [1]:
import gc
from pathlib import Path
import warnings

import anndata as ad
import scanpy as sc
import dask.array as da
import numpy as np
import pandas as pd
from statsmodels.stats.multitest import fdrcorrection
from tqdm.auto import tqdm

import cellink as cl
from cellink._core import DAnn, GAnn
from cellink.at.gwas import GWAS
from cellink.utils import column_normalize, gaussianize
from cellink.datasets import get_onek1k

In [2]:
n_gpcs = 20
n_epcs = 15
batch_e_pcs_n_top_genes = 2000
chrom = 22
cis_window = 500_000
cell_type = "CD8 Naive"
celltype_key = "predicted.celltype.l2"
pb_gex_key = f"PB_{cell_type}"  # pseudobulk expression in dd.G.obsm[key_added]
original_donor_col = "donor_id"
min_percent_donors_expressed = 0.1
do_debug = False

## Load and Prepare Data

Here, we load a prepared dataset (`onek1k`) that includes genotype and expression information from human donors. We also extract gene annotations using Ensembl via `pybiomart`, which are essential for defining cis-windows during eQTL analysis.

In [3]:
dd = get_onek1k(config_path='../../src/cellink/datasets/config/onek1k.yaml')
dd

/Users/larnoldt/cellink_sample_data/onek1k/onek1k_cellxgene.h5ad already exists. Verifying checksum.
/Users/larnoldt/cellink_sample_data/onek1k/OneK1K.noGP.vcf.gz already exists. Verifying checksum.
/Users/larnoldt/cellink_sample_data/onek1k/OneK1K.noGP.vcf.gz.csi already exists. Verifying checksum.
/Users/larnoldt/cellink_sample_data/onek1k/gene_counts_Ensembl_105_phenotype_metadata.tsv.gz already exists. Verifying checksum.


/opt/miniconda3/envs/single_cell_base3/lib/python3.12/site-packages/pandas/core/internals/blocks.py:2588: RuntimeWarning: invalid value encountered in cast
  return self.values.astype(_dtype_obj)


╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │      DonorData(981 x n_donors=10,595,884, n_cells_per_donor=[333-3,511], donor_id = 'donor_id')  │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 ×         │ AnnData object with n_obs × n_vars = 1,248,980 ×   │
│ 10,595,884                                         │ 36,469                                             │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     obs: 'orig.ident', 'nCount_RNA',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'pool_number', 'predicted.celltype.l2',            │
│ 'TYPED', 'TYPED_ONLY', 'id', 'id_mask', 'length',  │ 'predicted.celltype.l2.score', 'age',              │
│ 'quality', 'pos_hg19', 'id_hg19'                   │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     uns: 'kinship'                                 │     var: 'vst.mean', 'vst.variance',               │
│                                                    │ 'vst.variance.expected',                           │
│                                                    │ 'vst.variance.standardized', 'vst.variable',       │
│                                                    │ 'feature_is_filtered', 'feature_name',             │
│                                                    │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type'                   │
│     obsm: 'gPCs'                                   │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title'      │
│     varm: 'filter'                                 │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'                     │
│                                                    │     varm: 'PCs'                                    │
└────────────────────────────────────────────────────┴─────────────────────────────────────────

In [6]:
def _get_ensembl_gene_id_start_end_chr():
    from pybiomart import Server
    server = Server(host='http://www.ensembl.org')
    dataset = (server.marts['ENSEMBL_MART_ENSEMBL'].datasets['hsapiens_gene_ensembl'])
    ensembl_gene_id_start_end_chr = dataset.query(attributes=['ensembl_gene_id', 'start_position', 'end_position', 'chromosome_name'])
    ensembl_gene_id_start_end_chr = ensembl_gene_id_start_end_chr.set_index("Gene stable ID")
    ensembl_gene_id_start_end_chr = ensembl_gene_id_start_end_chr.rename(columns={
        "Gene start (bp)": GAnn.start,
        "Gene end (bp)": GAnn.end,
        "Chromosome/scaffold name": GAnn.chrom,
    })
    return ensembl_gene_id_start_end_chr

In [7]:
ensembl_gene_id_start_end_chr = _get_ensembl_gene_id_start_end_chr()
ensembl_gene_id_start_end_chr

,start,end,chrom
Gene stable ID,,,
ENSG00000210049,577,647,MT
ENSG00000211459,648,1601,MT
ENSG00000210077,1602,1670,MT
ENSG00000210082,1671,3229,MT
ENSG00000209082,3230,3304,MT
...,...,...,...
ENSG00000197989,28578536,28583132,1
ENSG00000229388,28643159,28648829,1
ENSG00000289291,28736044,28737670,1


In [8]:
dd.C.var = dd.C.var.join(ensembl_gene_id_start_end_chr)
dd.C.obs[DAnn.donor] = dd.C.obs[original_donor_col]
dd.G.obsm["gPCs"] = dd.G.obsm["gPCs"][dd.G.obsm["gPCs"].columns[:n_gpcs]]

## Normalize and Filter Expression Data

We normalize single-cell expression using total-count normalization and log transformation. We then filter the dataset to only include cells of a specific type (`CD8 Naive`). This step ensures we're analyzing homogeneous populations, which increases power in eQTL detection.

In [9]:
sc.pp.normalize_total(dd.C)
sc.pp.log1p(dd.C)
sc.pp.normalize_total(dd.C)

mdata = sc.get.aggregate(dd.C, by=DAnn.donor, func="mean")
mdata.X = mdata.layers.pop("mean")

sc.pp.highly_variable_genes(mdata, n_top_genes=batch_e_pcs_n_top_genes)
sc.tl.pca(mdata, n_comps=n_epcs)

dd.G.obsm["ePCs"] = mdata[dd.G.obs_names].obsm["X_pca"]

In [10]:
dd = dd[..., dd.C.obs[celltype_key] == cell_type, :].copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │        DonorData(980 x n_donors=10,595,884, n_cells_per_donor=[1-583], donor_id = 'donor_id')    │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 980 ×         │ AnnData object with n_obs × n_vars = 52,538 ×      │
│ 10,595,884                                         │ 36,469                                             │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     obs: 'orig.ident', 'nCount_RNA',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'pool_number', 'predicted.celltype.l2',            │
│ 'TYPED', 'TYPED_ONLY', 'id', 'id_mask', 'length',  │ 'predicted.celltype.l2.score', 'age',              │
│ 'quality', 'pos_hg19', 'id_hg19'                   │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     uns: 'kinship'                                 │     var: 'vst.mean', 'vst.variance',               │
│                                                    │ 'vst.variance.expected',                           │
│                                                    │ 'vst.variance.standardized', 'vst.variable',       │
│                                                    │ 'feature_is_filtered', 'feature_name',             │
│                                                    │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'start', 'end',  │
│                                                    │ 'chrom'                                            │
│     obsm: 'gPCs', 'ePCs'                           │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title',     │
│                                                    │ 'log1p'                                            │
│     varm: 'filter'                                 │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'          

In [11]:
gc.collect()

230

## Pseudobulk Aggregation and ePC Calculation

We compute pseudobulk expression profiles by averaging expression across all cells from the same donor. From these profiles, we identify highly variable genes and compute expression PCs (ePCs), which will be used as covariates in the regression model to control for expression heterogeneity.

In [12]:
dd.aggregate(key_added=pb_gex_key, sync_var=True, verbose=True)
dd.aggregate(obs=["sex", "age"], func="first", add_to_obs=True)
dd

INFO:cellink._core.donordata:Aggregated X to PB_CD8 Naive
INFO:cellink._core.donordata:Observation found for 980 donors.


╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │        DonorData(980 x n_donors=10,595,884, n_cells_per_donor=[1-583], donor_id = 'donor_id')    │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 980 ×         │ AnnData object with n_obs × n_vars = 52,538 ×      │
│ 10,595,884                                         │ 36,469                                             │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│                                                    │ 'pool_number', 'predicted.celltype.l2',            │
│                                                    │ 'predicted.celltype.l2.score', 'age',              │
│                                                    │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     var: 'vst.mean', 'vst.variance',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'vst.variance.expected',                           │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'vst.variance.standardized', 'vst.variable',       │
│ 'TYPED', 'TYPED_ONLY', 'id', 'id_mask', 'length',  │ 'feature_is_filtered', 'feature_name',             │
│ 'quality', 'pos_hg19', 'id_hg19'                   │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'start', 'end',  │
│                                                    │ 'chrom'                                            │
│     uns: 'kinship'                                 │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title',     │
│                                                    │ 'log1p'                                            │
│     obsm: 'gPCs', 'ePCs', 'PB_CD8 Naive'           │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'          

## Filter Genes Based on Donor Expression Coverage

To ensure statistical robustness, we exclude genes that are not expressed in at least 10% of donors. This prevents unreliable associations driven by low expression or sparse data.

In [14]:
print(f"{pb_gex_key} shape:", dd.G.obsm[pb_gex_key].shape)
print("dd.shape:", dd.shape)

keep_genes = ((dd.G.obsm[pb_gex_key] > 0).mean(axis=0) >= min_percent_donors_expressed).values
dd = dd[..., keep_genes]
print("after filtering")
print(f"{pb_gex_key} shape:", dd.G.obsm[pb_gex_key].shape)
print("dd.shape:", dd.shape)

PB_CD8 Naive shape: (980, 36469)
dd.shape: (980, 10595884, 52538, 36469)
after filtering
PB_CD8 Naive shape: (980, 12642)
dd.shape: (980, 10595884, 52538, 12642)


## Prepare Covariate Matrix

We construct a covariate matrix including sex, age, genotype PCs (gPCs), and expression PCs (ePCs). These help control for confounding effects in the association analysis. Covariates are normalized before being used in the regression model.

In [15]:
F = np.concatenate(
    [
        np.ones((dd.shape[0], 1)),
        dd.G.obs[["sex"]].values - 1,
        dd.G.obs[["age"]].values,
        dd.G.obsm["gPCs"].values,
        dd.G.obsm["ePCs"],
    ],
    axis=1,
)
F[:, 2:] = column_normalize(F[:, 2:])

## Run cis-eQTL Mapping

We perform linear regression for each gene against all SNPs within a ±500kb cis-window. For each gene, we retain the top associated SNP and record statistics such as p-value, beta, and likelihood ratio test (LRT) values.

In [16]:
# alternative to dd[:, dd.G.var.chrom == str(chrom), :, dd.C.var.chrom == str(chrom)]
dd = dd.sel(G_var=dd.G.var.chrom == str(chrom), C_var=dd.C.var.chrom == str(chrom)).copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │         DonorData(980 x n_donors=136,776, n_cells_per_donor=[1-583], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 980 × 136,776 │ AnnData object with n_obs × n_vars = 52,538 × 317  │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'percent.mt', 'donor_id',          │
│                                                    │ 'pool_number', 'predicted.celltype.l2',            │
│                                                    │ 'predicted.celltype.l2.score', 'age',              │
│                                                    │ 'organism_ontology_term_id',                       │
│                                                    │ 'tissue_ontology_term_id',                         │
│                                                    │ 'assay_ontology_term_id',                          │
│                                                    │ 'disease_ontology_term_id',                        │
│                                                    │ 'cell_type_ontology_term_id',                      │
│                                                    │ 'self_reported_ethnicity_ontology_term_id',        │
│                                                    │ 'development_stage_ontology_term_id',              │
│                                                    │ 'sex_ontology_term_id', 'is_primary_data',         │
│                                                    │ 'suspension_type', 'tissue_type', 'cell_type',     │
│                                                    │ 'assay', 'disease', 'organism', 'sex', 'tissue',   │
│                                                    │ 'self_reported_ethnicity', 'development_stage',    │
│                                                    │ 'observation_joinid'                               │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AC',         │     var: 'vst.mean', 'vst.variance',               │
│ 'AC_Hemi', 'AC_Het', 'AC_Hom', 'AF', 'AN', 'ER2',  │ 'vst.variance.expected',                           │
│ 'ExcHet', 'HWE', 'IMPUTED', 'maf', 'NS', 'R2',     │ 'vst.variance.standardized', 'vst.variable',       │
│ 'TYPED', 'TYPED_ONLY', 'id', 'id_mask', 'length',  │ 'feature_is_filtered', 'feature_name',             │
│ 'quality', 'pos_hg19', 'id_hg19'                   │ 'feature_reference', 'feature_biotype',            │
│                                                    │ 'feature_length', 'feature_type', 'start', 'end',  │
│                                                    │ 'chrom'                                            │
│     uns: 'kinship'                                 │     uns: 'cell_type_ontology_term_id_colors',      │
│                                                    │ 'citation', 'default_embedding',                   │
│                                                    │ 'schema_reference', 'schema_version', 'title',     │
│                                                    │ 'log1p'                                            │
│     obsm: 'gPCs', 'ePCs', 'PB_CD8 Naive'           │     obsm: 'X_azimuth_spca', 'X_azimuth_umap',      │
│                                                    │ 'X_harmony', 'X_pca', 'X_umap'                     │
│     varm: 'filter'                                 │     varm: 'PCs'                         

In [17]:
results = []
if isinstance(dd.G.X, da.Array | ad._core.views.DaskArrayView):
    if dd.G.is_view:
        dd._G = dd._G.copy()  # TODO: discuss with SWEs
    dd.G.X = dd.G.X.compute()

if do_debug:
    warnings.filterwarnings("ignore", category=RuntimeWarning)

for gene, row in tqdm(dd.C.var.iterrows(), total=dd.shape[3]):
    Y = gaussianize(dd.G.obsm[pb_gex_key][[gene]].values + 1e-5 * np.random.randn(dd.shape[0], 1))

    start = max(0, row.start - cis_window)
    end = row.end + cis_window
    _G = dd.G[:, (dd.G.var.pos < end)]
    _G = _G[:, (_G.var.pos > start)]
    _G = _G[:, (_G.X.std(0) != 0)]
    G = _G.X

    if G.shape[1] > 0:
        gwas = GWAS(Y, F)
        gwas.process(G)

        snp_idx = gwas.getPv().argmin()

        def _get_top_snp(arr, snp_idx=snp_idx):
            return arr.ravel()[snp_idx].item()

        rdict = {
            "snp": _G.var.iloc[snp_idx].name,
            "egene": gene,
            "n_cis_snps": G.shape[1],
            "pv": _get_top_snp(gwas.getPv()),
            "beta": _get_top_snp(gwas.getBetaSNP()),
            "betaste": _get_top_snp(gwas.getBetaSNPste()),
            "lrt": _get_top_snp(gwas.getLRT()),
        }
        results.append(rdict)

rdf = pd.DataFrame(results)
rdf

  0%|          | 0/317 [00:00<?, ?it/s]

,snp,egene,n_cis_snps,pv,beta,betaste,lrt
0,22_17314518_A_G,ENSG00000177663,4389,0.001254,-0.188865,0.058539,10.409214
1,22_17219674_T_G,ENSG00000069998,4652,0.000110,0.746762,0.193087,14.957528
2,22_17360842_TCAAACAAA_T,ENSG00000093072,5111,0.001117,-0.313230,0.096105,10.622749
3,22_17175008_A_T,ENSG00000131100,5410,0.000155,-0.167189,0.044197,14.309812
4,22_17874782_G_A,ENSG00000099968,5502,0.000215,0.522235,0.141135,13.691846
...,...,...,...,...,...,...,...
311,22_50139875_C_T,ENSG00000205560,3605,0.000181,-0.178023,0.047551,14.016494
312,22_50608430_A_G,ENSG00000100288,3528,0.000044,-0.855895,0.209371,16.711213
313,22_50632488_C_G,ENSG00000205559,3501,0.000364,-0.462915,0.129841,12.711048
314,22_50590568_CTTT_C,ENSG00000100299,3254,0.001375,0.529866,0.165596,10.238363


## Collect and Adjust Results

We collect all results into a DataFrame and perform multiple testing correction. The Benjamini-Hochberg FDR (`qv`) is computed to identify statistically significant gene-SNP pairs.

We then count the number of genes with significant associations (FDR < 0.05).

In [18]:
rdf["pv_adj"] = np.clip(rdf["pv"] * rdf["n_cis_snps"], 0, 1)  # gene-wise Bonferroni
rdf["qv"] = fdrcorrection(rdf["pv_adj"])[1]

(rdf.qv < 0.05).sum()

13

In [19]:
rdf.to_csv("pseudobulk_eqtl_cr22.csv")